In [ ]:
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.14: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Add", # instruction
        "55589+6664317+8897564+893+1562348+847147", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<｜begin▁of▁sentence｜>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Add

### Input:
55589+6664317+8897564+893+1562348+847147

### Response:
I need to calculate the sum of these numbers: 55589, 6664317, 8897564, 893, 

KeyboardInterrupt: 

### Data Preprocessing

In [ ]:
import json
from datasets import Dataset

# Load your JSON file
with open("/kaggle/input/llm-adition/output.json", "r", encoding="utf-8") as f:
    data = json.load(f)



In [ ]:
instructions = []
inputs = []
outputs = []

for entry in data:

    nums = entry.get('numbers')
    original_thought = entry.get('thought')
    new_thought = entry.get('new_thought')
    ans = entry.get('actual_answer')


    textin =f"Add the numbers given: {nums} <think>{original_thought}"
    textout = f'{new_thought}</think><answer>{ans}</answer>'


    instructions.append('')
    inputs.append(textin)
    outputs.append(textout)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}
### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token


texts = []
for instruction, input, output in zip(instructions, inputs, outputs):
    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    texts.append(text)




In [ ]:
result = { "text" : texts }
print(result)

{'text': ["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\n\n### Input:\nAdd the numbers given: [4402730, 1256753, 9380747, 757185, 1652619, 8367780] <think>First, I need to add all the given numbers: 4,402,730 + 1,256,753 + 9,380,747 + 757,185 + 1,652,619 + 8,367,780.\n\nTo simplify the calculation, I'll group numbers by their place values to make addition easier.\n\nAdding all the millions together: 4,000,000 + 1,000,000 + 9,000,000 = 14,000,000.\n\nNext, adding all the hundred thousands: 400,000 + 200,000 + 300,000 = 900,000.\n\nThen, for the ten thousands place: 7,000 + 5,000 + 8,000 = 21,000.\n\nIn the thousands place: 700 + 600 + 700 = 2,000.\n\nFor the hundreds place: 30 + 70 + 40 = 140.\n\nFinally, in the tens and ones places, all numbers have zero, so they contribute nothing to the sum.\n\nNow, adding all these together: 14,000,000 + 900,000 + 21

In [ ]:
print(texts[0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:


### Input:
Add the numbers given: [4402730, 1256753, 9380747, 757185, 1652619, 8367780] <think>First, I need to add all the given numbers: 4,402,730 + 1,256,753 + 9,380,747 + 757,185 + 1,652,619 + 8,367,780.

To simplify the calculation, I'll group numbers by their place values to make addition easier.

Adding all the millions together: 4,000,000 + 1,000,000 + 9,000,000 = 14,000,000.

Next, adding all the hundred thousands: 400,000 + 200,000 + 300,000 = 900,000.

Then, for the ten thousands place: 7,000 + 5,000 + 8,000 = 21,000.

In the thousands place: 700 + 600 + 700 = 2,000.

For the hundreds place: 30 + 70 + 40 = 140.

Finally, in the tens and ones places, all numbers have zero, so they contribute nothing to the sum.

Now, adding all these together: 14,000,000 + 900,000 + 21,000 + 2,000 + 140 = 14,923,140.


In [ ]:
from datasets import Dataset

# Assuming 'result' contains your formatted prompts
prompts = result["text"]

# Create a simple dataset
dataset = Dataset.from_dict({"text": prompts})

Tokenized '### Response:': ['Ċ', '###', 'ĠResponse', ':Ċ']


In [ ]:
from transformers import DataCollatorForLanguageModeling
import torch

class CustomDataCollator(DataCollatorForLanguageModeling):
    def __init__(self, response_template, tokenizer, *args, **kwargs):
        super().__init__(tokenizer=tokenizer, mlm=False, *args, **kwargs)
        self.response_template = response_template
        self.response_token_ids = tokenizer.encode(
            response_template,
            add_special_tokens=False,
        )

    def torch_call(self, examples):
        batch = super().torch_call(examples)
        labels = batch["labels"].clone()

        for i in range(len(examples)):
            input_ids = examples[i]["input_ids"]

            if isinstance(input_ids, torch.Tensor):
                input_ids = input_ids.tolist()

            response_start = None
            for j in range(len(input_ids) - len(self.response_token_ids) + 1):

                if input_ids[j:j+len(self.response_token_ids)] == self.response_token_ids:
                    response_start = j + len(self.response_token_ids)
                    break

            if response_start is not None:
                labels[i, :response_start] = -100

        batch["labels"] = labels
        return batch

In [ ]:
response_template = " Response"
collator = CustomDataCollator(response_template, tokenizer=tokenizer)

# Test on a single example
example = dataset[0]["text"]

tokenized = tokenizer(
    example,
    return_tensors="pt",
    add_special_tokens=False,
    truncation=False,
    padding=False,
)

batch = collator([{"input_ids": tokenized["input_ids"][0]}])
labels = batch["labels"]

print("Labels:", labels)


Labels: tensor([[  -100,   -100,   -100,  ...,   9217,     29, 151643]])


## FT

In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

## calculate loss only on response tokens
response_template = " Response"
collator = CustomDataCollator(response_template, tokenizer=tokenizer)




trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    data_collator=collator,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 10,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 40,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_hf",
        weight_decay = 0.02,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1051 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,051 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 18,464,768/5,000,000,000 (0.37% trained)


Step,Training Loss
1,0.147100
2,0.221900
3,0.196700
4,0.203500
5,0.206400
6,0.187100
7,0.146500
8,0.158200
9,0.170900
10,0.193800


In [ ]:
model.save_pretrained("deepseek_qwen1.5-ftnew")
tokenizer.save_pretrained("deepseek_qwen1.5-ftnew")

('deepseek_qwen1.5-ftnew/tokenizer_config.json',
 'deepseek_qwen1.5-ftnew/special_tokens_map.json',
 'deepseek_qwen1.5-ftnew/tokenizer.json')

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "deepseek_qwen1.5-ftnew",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "add the numbers given. Reason and check for correctness", # instruction
        "4402730, 1256753, 9380747, 757185, 1652619, 8367780", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<｜begin▁of▁sentence｜>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
add the numbers given. Reason and check for correctness

### Input:
4402730, 1256753, 9380747, 757185, 1652619, 8367780
### Response:
The numbers to add are 4402730, 1256753, 9380747, 757185, 1652619, and 8367780. Let's perform the addition step by step.

First, add 4402730 and 1256753:
4402730 + 1256753 = 5659483

Next, add the result from the first step to 9380747:
5659483 + 9380747 = 15040230

Then, add the result from the second step to 757185:
15040230 + 757185 = 15797415

Next, add the result from the third step to 1652619:
15797415 + 1652619 = 17449034

Finally, add the result from the fourth step to 8367780:
17449034 + 8367780 = 25816814

The sum of the numbers is 25816814.

The sum of the numbers is 25816814. The sum is 25816814, which is greater than the original numbers. Let's veri

In [ ]:
print(sum([4402730, 1256753, 9380747, 757185, 1652619, 8367780]))

25817814
